Interrogating Returns Distribution Data for Stock Data
---

In [1]:
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

data = pdr.get_data_yahoo("msft aapl", start="2022-01-01", end="2022-12-31")
display(data.head())

[*********************100%%**********************]  2 of 2 completed


Adj Close                   Close                    High  \
                  AAPL        MSFT        AAPL        MSFT        AAPL   
Date                                                                     
2022-01-03  179.953888  328.727722  182.009995  334.750000  182.880005   
2022-01-04  177.669998  323.090942  179.699997  329.010010  182.940002   
2022-01-05  172.943985  310.688171  174.919998  316.380005  180.169998   
2022-01-06  170.056961  308.233154  172.000000  313.880005  175.300003   
2022-01-07  170.225067  308.390259  172.169998  314.040009  174.139999   

                               Low                    Open              \
                  MSFT        AAPL        MSFT        AAPL        MSFT   
Date                                                                     
2022-01-03  338.000000  177.710007  329.779999  177.830002  335.350006   
2022-01-04  335.200012  179.119995  326.119995  182.630005  334.829987   
2022-01-05  326.070007  174.639999  315.980011  179.610001  325.859985   
2022-01-06  318.700012  171.639999  311.489990  172.699997  313.149994   
2022-01-07  316.500000  171.029999  310.089996  172.889999  314.149994   

               Volume            
                 AAPL      MSFT  
Date                             
2022-01-03  104487900  28865100  
2022-01-04   99310400  32674300  
2022-01-05   94537600  40054300  
2022-01-06   96904000  39646100  
2022-01-07   86709100  32720000

In [2]:
import pandas as pd

tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers.head())

  Symbol     Security             GICS Sector               GICS Sub-Industry  \
0    MMM           3M             Industrials        Industrial Conglomerates   
1    AOS  A. O. Smith             Industrials               Building Products   
2    ABT       Abbott             Health Care           Health Care Equipment   
3   ABBV       AbbVie             Health Care                   Biotechnology   
4    ACN    Accenture  Information Technology  IT Consulting & Other Services   

     Headquarters Location  Date added      CIK      Founded  
0    Saint Paul, Minnesota  1957-03-04    66740         1902  
1     Milwaukee, Wisconsin  2017-07-26    91142         1916  
2  North Chicago, Illinois  1957-03-04     1800         1888  
3  North Chicago, Illinois  2012-12-31  1551152  2013 (1888)  
4          Dublin, Ireland  2011-07-06  1467373         1989  


In [3]:
data = pdr.get_data_yahoo(tickers.Symbol.to_list(), start="2016-01-01", end="2016-12-31")
data.head()

[*********************100%%**********************]  503 of 503 completed


18 Failed downloads:
['CARR', 'INVH', 'FOXA', 'VICI', 'KVUE', 'CTVA', 'CEG', 'CDAY', 'MRNA', 'ABNB', 'VLTO', 'DOW', 'FOX', 'IR', 'OTIS', 'GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 1451624400, endDate = 1483160400")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-01-01 -> 2016-12-31)')


Adj Close                                        \
                             A        AAL       AAPL       ABBV ABNB   
Date                                                                   
2016-01-04 00:00:00  38.198948  39.095455  23.977478  40.778076  NaN   
2016-01-05 00:00:00  38.067505  38.722759  23.376614  40.608192  NaN   
2016-01-06 00:00:00  38.236485  39.401264  22.919146  40.615280  NaN   
2016-01-07 00:00:00  36.612396  38.655872  21.951851  40.494934  NaN   
2016-01-08 00:00:00  36.227505  38.579411  22.067926  39.390728  NaN   

                                                                            \
                           ABT       ACGL        ACN       ADBE        ADI   
Date                                                                         
2016-01-04 00:00:00  36.956322  22.950001  89.603127  91.970001  46.181301   
2016-01-05 00:00:00  36.947701  23.033333  90.069481  92.339996  45.841988   
2016-01-06 00:00:00  36.637791  23.070000  89.893501  91.019997  43.890907   
2016-01-07 00:00:00  35.759724  23.046667  87.253700  89.110001  42.771160   
2016-01-08 00:00:00  35.010780  22.806667  86.408981  87.849998  42.397896   

                     ...   Volume                                       \
                     ...     WYNN      XEL       XOM     XRAY      XYL   
Date                 ...                                                 
2016-01-04 00:00:00  ...  4249800  2819300  20400100  1387100  1353400   
2016-01-05 00:00:00  ...  4286100  2141200  11993500  1611200  1075400   
2016-01-06 00:00:00  ...  3316100  5191900  18826900  1179700  1866000   
2016-01-07 00:00:00  ...  5525600  5212200  21263800  1140400  1555400   
2016-01-08 00:00:00  ...  5740000  3005400  19033600  1233700  1471500   

                                                                   
                          YUM      ZBH     ZBRA     ZION      ZTS  
Date                                                               
2016-01-04 00:00:00   4821623  2104084   656200  3523300  2871700  
2016-01-05 00:00:00   5273420  2294737   611300  2653600  3116700  
2016-01-06 00:00:00   5049330  2751233   881900  4164200  4670500  
2016-01-07 00:00:00  11005453  1829589  1210600  4234000  4324400  
2016-01-08 00:00:00   5996184  1670969  1034700  3401600  4946100  

[5 rows x 3018 columns]

Finding log returns for stock data

In [4]:
import numpy as np

returns = np.log1p(data["Adj Close"].pct_change())
for name in returns.columns:
    data[("Daily Log Returns",name)] = returns[name]
returns.head()

/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/804343652.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Adj Close"].pct_change())
/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/804343652.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[("Daily Log Returns",name)] = returns[name]


,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05 00:00:00,-0.003447,-0.009579,-0.025379,-0.004175,NaN,-0.000233,0.003624,0.005191,0.004015,-0.007375,...,0.018727,0.010033,0.008485,0.020182,-0.000278,-0.002496,0.020610,-0.025437,-0.010917,0.015533
2016-01-06 00:00:00,0.004429,0.017370,-0.019763,0.000175,NaN,-0.008423,0.001591,-0.001956,-0.014398,-0.043493,...,-0.053196,0.010483,-0.008355,-0.014591,-0.012554,-0.007106,0.004320,-0.040777,-0.029578,0.000209
2016-01-07 00:00:00,-0.043403,-0.019099,-0.043121,-0.002967,NaN,-0.024258,-0.001012,-0.029806,-0.021208,-0.025843,...,-0.098793,0.003834,-0.016135,-0.008824,-0.026167,-0.034712,-0.023261,-0.046375,-0.030480,-0.030876
2016-01-08 00:00:00,-0.010568,-0.001980,0.005274,-0.027646,NaN,-0.021166,-0.010468,-0.009728,-0.014241,-0.008765,...,-0.041556,-0.010995,-0.020409,-0.029053,-0.009556,-0.013701,-0.004226,-0.002697,-0.011318,-0.014712


In [6]:
#Drop symbols with less than 200 datapoints of returns
returns = returns.dropna(axis=1, thresh=len(returns)-200)
returns.tail()

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-23 00:00:00,0.004991,-0.004322,0.001976,0.010968,0.003389,0.001835,-0.002635,0.002861,0.005295,-0.000664,...,0.001019,0.000000,-0.001762,0.001859,-0.002350,-0.003754,0.007357,0.002897,0.002740,0.009716
2016-12-27 00:00:00,0.006688,0.002678,0.006331,0.002563,0.004674,-0.006324,0.000596,-0.000381,0.006346,0.011014,...,-0.001133,0.001474,0.000441,-0.002874,-0.002945,0.003442,0.007982,0.007264,0.001595,-0.001116
2016-12-28 00:00:00,-0.017132,-0.019527,-0.004273,-0.003687,-0.009893,-0.008107,-0.008029,-0.011593,-0.010144,-0.006814,...,-0.013233,-0.010862,-0.004971,-0.014376,-0.018254,-0.004853,0.001554,-0.016681,-0.014907,-0.005226
2016-12-29 00:00:00,0.001140,-0.008849,-0.000257,0.007360,0.002352,0.001046,0.003425,-0.000868,0.000272,0.000441,...,0.001033,0.016004,0.000553,0.001718,-0.003410,-0.001413,0.004260,0.003847,-0.011386,0.003363
2016-12-30 00:00:00,-0.001754,-0.011923,-0.007826,-0.001755,0.002607,0.002320,0.001025,-0.007066,-0.013133,0.006373,...,-0.007600,-0.005879,-0.000996,-0.008967,-0.005036,-0.004726,-0.002903,-0.002097,0.005826,-0.001680


In [7]:
data.head()

Adj Close                                        \
                             A        AAL       AAPL       ABBV ABNB   
Date                                                                   
2016-01-04 00:00:00  38.198948  39.095455  23.977478  40.778076  NaN   
2016-01-05 00:00:00  38.067505  38.722759  23.376614  40.608192  NaN   
2016-01-06 00:00:00  38.236485  39.401264  22.919146  40.615280  NaN   
2016-01-07 00:00:00  36.612396  38.655872  21.951851  40.494934  NaN   
2016-01-08 00:00:00  36.227505  38.579411  22.067926  39.390728  NaN   

                                                                            \
                           ABT       ACGL        ACN       ADBE        ADI   
Date                                                                         
2016-01-04 00:00:00  36.956322  22.950001  89.603127  91.970001  46.181301   
2016-01-05 00:00:00  36.947701  23.033333  90.069481  92.339996  45.841988   
2016-01-06 00:00:00  36.637791  23.070000  89.893501  91.019997  43.890907   
2016-01-07 00:00:00  35.759724  23.046667  87.253700  89.110001  42.771160   
2016-01-08 00:00:00  35.010780  22.806667  86.408981  87.849998  42.397896   

                     ... Daily Log Returns                                \
                     ...              WYNN       XEL       XOM      XRAY   
Date                 ...                                                   
2016-01-04 00:00:00  ...               NaN       NaN       NaN       NaN   
2016-01-05 00:00:00  ...          0.018727  0.010033  0.008485  0.020182   
2016-01-06 00:00:00  ...         -0.053196  0.010483 -0.008355 -0.014591   
2016-01-07 00:00:00  ...         -0.098793  0.003834 -0.016135 -0.008824   
2016-01-08 00:00:00  ...         -0.041556 -0.010995 -0.020409 -0.029053   

                                                                       \
                          XYL       YUM       ZBH      ZBRA      ZION   
Date                                                                    
2016-01-04 00:00:00       NaN       NaN       NaN       NaN       NaN   
2016-01-05 00:00:00 -0.000278 -0.002496  0.020610 -0.025437 -0.010917   
2016-01-06 00:00:00 -0.012554 -0.007106  0.004320 -0.040777 -0.029578   
2016-01-07 00:00:00 -0.026167 -0.034712 -0.023261 -0.046375 -0.030480   
2016-01-08 00:00:00 -0.009556 -0.013701 -0.004226 -0.002697 -0.011318   

                               
                          ZTS  
Date                           
2016-01-04 00:00:00       NaN  
2016-01-05 00:00:00  0.015533  
2016-01-06 00:00:00  0.000209  
2016-01-07 00:00:00 -0.030876  
2016-01-08 00:00:00 -0.014712  

[5 rows x 3521 columns]

Creating dataframe with distribution data

In [8]:
#Using scipy.stats skew and kurtosis, dropping any nan values from measurement
import scipy.stats as stats

symbols = returns.columns
sample_stats = pd.DataFrame({"Observations": (len(returns[symbol]) for symbol in symbols),
                      "Mean Return": list(np.mean(returns, axis=0)),
                      "Standard Deviation": list(np.std(returns, axis=0)),
                      "Skewness": list(stats.skew(returns, axis=0, nan_policy='omit')),
                      "Kurtosis": list(stats.kurtosis(returns, axis=0, nan_policy='omit')),
                      "K-Squared P-Value": list(stats.normaltest(returns, axis=0, nan_policy='omit').pvalue)},
                      index = symbols)
sample_stats.head()

,Observations,Mean Return,Standard Deviation,Skewness,Kurtosis,K-Squared P-Value
A,252,0.000494,0.014772,-0.822091,1.961756,4.789478e-09
AAL,252,0.000571,0.023052,-0.379115,3.623910,6.493222e-08
AAPL,252,0.000465,0.014723,-0.320077,4.301049,1.748651e-08
ABBV,252,0.000483,0.017080,0.016762,3.090482,7.970633e-06
ABT,252,-0.000344,0.015094,-1.907165,9.541310,2.818503e-29


In [9]:
measured = sum(sample_stats["K-Squared P-Value"] != np.nan)
non_normal = len(sample_stats[sample_stats["K-Squared P-Value"] < 0.05])
print(f"Of {measured} S&P 500 stocks with a measurable K-Squared Statistic, \
\n{non_normal} have sufficiently low P-Value to reject the null hypothesis\
\nthat their returns are normally distributed")

Of 483 S&P 500 stocks with a measurable K-Squared Statistic, 
477 have sufficiently low P-Value to reject the null hypothesis
that their returns are normally distributed


*Function to find returns distribution for stock data, and test hypothesis of normally distributed returns with constant variance*

In [10]:
def returns_distribution(tickers: list[str], start: str, end:str, filename=None, header=None):
    """
    Finds returns distribution for stock data, and tests hypothesis of normally distributed returns
    with constant variance
    :param tickers: The ticker symbol for data to be downloaded
    :param start: The start date for downloaded stock data
    :param end: The end date for downloaded stock data
    :param filename: The name of the file for the data to be uploaded to (optional)
    :param header: The header configuration for the file (optional)
    :return: Dataframe with the number of observations, mean return, standard deviation of returns,
    skewness of returns, kurtosis of returns, and k-squared test p-value of the returns
    """
    data = yf.download(tickers, start=start, end=end)
    returns = np.log1p(data["Close"].pct_change())
    returns = returns.dropna(axis=1, thresh=len(returns)-200)
    
    symbols = returns.columns
    sample_stats = pd.DataFrame({"Observations": (len(returns[symbol]) for symbol in symbols),
                      "Mean Return": list(np.mean(returns, axis=0)),
                      "Standard Deviation": list(np.std(returns, axis=0)),
                      "Skewness": list(stats.skew(returns, axis=0, nan_policy='omit')),
                      "Kurtosis": list(stats.kurtosis(returns, axis=0, nan_policy='omit')),
                      "K-Squared P-Value": list(stats.normaltest(returns, axis=0, nan_policy='omit').pvalue)},
                      index = symbols)
    
    if filename is not None:
        sample_stats.to_csv(filename, header=header)
    
    return sample_stats

In [11]:
import pandas as pd

tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

symbols = tickers.Symbol.to_list()
ss = returns_distribution(symbols, "2022-01-01", "2022-12-31", filename="rd-2022.csv")
ss.head()

[*********************100%%**********************]  503 of 503 completed


4 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-31)')


/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/3084458691.py:14: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Close"].pct_change())


,Observations,Mean Return,Standard Deviation,Skewness,Kurtosis,K-Squared P-Value
A,251,-0.000179,0.022148,0.308525,0.488630,3.953602e-02
AAL,251,-0.001552,0.035410,-0.030688,0.572806,2.124210e-01
AAPL,251,-0.001348,0.022387,0.231176,0.865158,2.134347e-02
ABBV,251,0.000707,0.014295,-0.882248,2.856754,3.825678e-11
ABNB,251,-0.002812,0.038346,-0.326138,0.434121,3.813268e-02


In [12]:
measured = non_normal = 0
for year in range(2000, 2023):
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    rd = returns_distribution(symbols, start, end, filename=f"returns_distribution_data/rd-{year}.csv")
    non_normal += sum(sample_stats["K-Squared P-Value"] < 0.05)
    measured += len(sample_stats)

[*********************100%%**********************]  503 of 503 completed


133 Failed downloads:
['FSLR', 'KHC', 'GPN', 'CARR', 'CZR', 'NWSA', 'HPE', 'LYV', 'INVH', 'CTLT', 'GOOG', 'PAYC', 'FLT', 'FTNT', 'ENPH', 'MA', 'TDG', 'FOXA', 'AIZ', 'LKQ', 'CMG', 'ALLE', 'CBRE', 'TRGP', 'TMUS', 'EXR', 'PYPL', 'VICI', 'PANW', 'CTVA', 'BR', 'KVUE', 'LVS', 'DAL', 'ABBV', 'ZBH', 'XYL', 'CEG', 'AWK', 'BG', 'CDAY', 'GM', 'WYNN', 'NDAQ', 'MRNA', 'CFG', 'KDP', 'CRM', 'ZTS', 'WBD', 'SEDG', 'PRU', 'MDLZ', 'CF', 'AAL', 'ABNB', 'CME', 'APTV', 'CBOE', 'CDW', 'ULTA', 'ELV', 'IQV', 'NCLH', 'HWM', 'KEYS', 'HCA', 'AXON', 'EPAM', 'MOH', 'VLTO', 'VRSK', 'DOW', 'TEL', 'AMCR', 'ACN', 'LW', 'DXCM', 'MSCI', 'CE', 'ICE', 'FOX', 'MKTX', 'CNC', 'UAL', 'PODD', 'FTV', 'WTW', 'GOOGL', 'LDOS', 'NXPI', 'TSLA', 'BX', 'PSX', 'PM', 'HLT', 'AVGO', 'FANG', 'QRVO', 'EXPE', 'NWS', 'NRG', 'ETSY', 'PFG', 'GNRC', 'DPZ', 'MPC', 'SYF', 'ANET', 'NOW', 'FIS', 'DLR', 'PARA', 'CHTR', 'DG', 'IR', 'WRK', 'HII', 'STX', 'OTIS', 'NFLX', 'ALGN', 'LYB', 'V', 'AMP', 'META', 'DFS', 'LULU', 'GEHC', 'MPWR', 'KMI']: Exception


[                       0%%                      ]  2 of 503 completed

/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/3084458691.py:14: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Close"].pct_change())


[*********************100%%**********************]  503 of 503 completed

...


6 Failed downloads:
['KVUE', 'CEG', 'VLTO', 'GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 1609477200, endDate = 1640926800")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-01 -> 2021-12-31)')


/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/3084458691.py:14: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Close"].pct_change())


[*********************100%%**********************]  503 of 503 completed


4 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-31)')


/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/3084458691.py:14: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Close"].pct_change())


In [13]:
print(f"From 2000 to 2022, of {measured} measured year long stock returns behaviour, \
\n{non_normal} showed behaviour sufficient to reject the null-hypothesis that they\
\nare normally distributed with constant variance, based on the K-Squared Test")

From 2000 to 2022, of 11109 measured year long stock returns behaviour, 
10971 showed behaviour sufficient to reject the null-hypothesis that they
are normally distributed with constant variance, based on the K-Squared Test


In [14]:
returns["AAPL"]

Date
2016-01-04         NaN
2016-01-05   -0.025379
2016-01-06   -0.019763
2016-01-07   -0.043121
2016-01-08    0.005274
                ...   
2016-12-23    0.001976
2016-12-27    0.006331
2016-12-28   -0.004273
2016-12-29   -0.000257
2016-12-30   -0.007826
Name: AAPL, Length: 252, dtype: float64

In [15]:
from statsmodels.tsa.stattools import acf, pacf
print(acf(returns["AAPL"], missing="conservative"))
print(acf(abs(returns["AAPL"]), missing="conservative"))

[ 1.          0.07476076 -0.0111054  -0.08317879 -0.05215588  0.01400959
  0.05787196  0.12701511  0.03059031  0.08775885 -0.0465251   0.01297636
 -0.05761752 -0.03579687  0.01146093 -0.04107294 -0.0205257  -0.01897049
 -0.02608329 -0.02787475  0.00880794 -0.0743006  -0.05691267 -0.05132423
  0.01881741]
[ 1.          0.11804963  0.1553612   0.09634534  0.11981463  0.06844406
 -0.01571357  0.12150808 -0.00923563 -0.00802013  0.07915942  0.07715719
 -0.07752164  0.14649487 -0.02886468  0.04135896 -0.01863158 -0.03975544
  0.04589823 -0.02981041  0.0562499   0.05215087 -0.02393289  0.01873019
 -0.09499949]


In [16]:
returns["AAPL"].autocorr(lag=2)

-0.011256042054929487

In [17]:
acf(abs(returns["AAPL"]).dropna(), nlags=5, qstat=True)

(array([1.        , 0.11804963, 0.1553612 , 0.09634534, 0.11981463,
        0.06844406]),
 array([ 3.53983869,  9.69557565, 12.07243785, 15.76320776, 16.97249833]),
 array([0.05991129, 0.00784571, 0.00713909, 0.00335393, 0.00455225]))

In [18]:
#Autocorrelation stats for stocks: autocorrelation, ljung-box q-statistic, p-value
def ac_format(ac):
    """
    Filters acf function to remove autocorrelation value for 0 lag to make array same dimension
    """
    lags = ac[0][1:6]
    rest = ac[1:3]
    return [lags] + list(rest)

full_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna(), nlags=5, qstat=True)) for symbol in returns.columns))
full_sqr_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna()**2, nlags=5, qstat=True)) for symbol in returns.columns))
full_abs_r_ac = np.array(list(ac_format(acf(abs(returns[symbol].dropna()), nlags=5, qstat=True)) for symbol in returns.columns))
r_pac = np.array(list(pacf(returns[symbol].dropna(), nlags=5)[1:6] for symbol in returns.columns)).T
sqr_r_pac = np.array(list(pacf(returns[symbol].dropna()**2, nlags=5)[1:6] for symbol in returns.columns)).T
abs_r_pac = np.array(list(pacf(abs(returns[symbol].dropna()), nlags=5)[1:6] for symbol in returns.columns)).T

In [19]:
#Slicing to get autocorrelations, q-statistic, and p-values exclusively
r_ac = full_r_ac[:,0].T
r_ldq = full_r_ac[:,1].T
r_pval = full_r_ac[:,2].T
sqr_r_ac = full_sqr_r_ac[:,0].T
sqr_r_ldq = full_sqr_r_ac[:,1].T
sqr_r_pval = full_sqr_r_ac[:,2].T
abs_r_ac = full_abs_r_ac[:,0].T
abs_r_ldq = full_abs_r_ac[:,1].T
abs_r_pval = full_abs_r_ac[:,2].T

In [20]:
#P-Value data-frame for different time lags
p = pd.DataFrame(
    {
    **{("Returns Autocorrelation P Value", f"Lag {l+1}"): r_pval[l] for l in range(5)},
    **{("Squared Returns Autocorrelation P Value", f"Lag {l+1}"): sqr_r_pval[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation P Value", f"Lag {l+1}"): abs_r_pval[l] for l in range(5)}
    }, index = returns.columns
)
p.head()

Returns Autocorrelation P Value                                          \
                               Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
A                           0.130921  0.268312  0.367428  0.530027  0.385199   
AAL                         0.357322  0.650913  0.720307  0.669926  0.581132   
AAPL                        0.233450  0.484041  0.358536  0.416645  0.553338   
ABBV                        0.164150  0.165971  0.064061  0.108301  0.081173   
ABT                         0.333690  0.522005  0.464737  0.626952  0.758816   

     Squared Returns Autocorrelation P Value                              \
                                       Lag 1         Lag 2         Lag 3   
A                                   0.050568  1.255181e-01  1.702617e-01   
AAL                                 0.010124  5.751074e-03  1.273537e-02   
AAPL                                0.544744  3.409333e-01  9.611677e-02   
ABBV                                0.000177  5.164326e-11  8.995405e-11   
ABT                                 0.110808  2.721920e-01  4.408785e-01   

                                 Absolute Returns Autocorrelation P Value  \
             Lag 4         Lag 5                                    Lag 1   
A     2.784908e-01  2.353496e-01                                 0.047793   
AAL   2.384037e-02  4.631031e-02                                 0.051396   
AAPL  1.554877e-01  2.137252e-01                                 0.059911   
ABBV  2.861434e-10  1.086147e-09                                 0.004960   
ABT   5.868604e-01  7.057702e-01                                 0.001523   

                                                              
             Lag 2         Lag 3         Lag 4         Lag 5  
A     1.402036e-01  2.385681e-01  2.641830e-01  8.410957e-02  
AAL   2.180922e-02  2.957690e-02  5.376955e-02  7.861911e-02  
AAPL  7.845714e-03  7.139094e-03  3.353933e-03  4.552251e-03  
ABBV  6.047782e-11  4.925164e-11  3.712702e-11  7.740550e-11  
ABT   6.259967e-03  1.685231e-02  3.586956e-02  6.647382e-02

In [21]:
#Ljung-Box Q-Statistic Dataframe for different time lags
ldq = pd.DataFrame(
    {
    **{("Returns Ljung-Box Q Statistic", f"Lag {l+1}"): r_ldq[l] for l in range(5)},
    **{("Squared Returns Ljung-Box Q Statistic", f"Lag {l+1}"): sqr_r_ldq[l] for l in range(5)},
    **{("Absolute Returns Ljung-Box Q Statistic", f"Lag {l+1}"): abs_r_ldq[l] for l in range(5)}
    }, index = returns.columns
)
ldq.head()

Returns Ljung-Box Q Statistic                                          \
                             Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
A                         2.281544  2.631208  3.161235  3.168539  5.258110   
AAL                       0.847284  0.858759  1.337249  2.359679  3.782524   
AAPL                      1.419716  1.451170  3.222778  3.922143  3.972808   
ABBV                      1.935571  3.591889  7.259830  7.578411  9.797740   
ABT                       0.934523  1.300158  2.558919  2.599271  2.616768   

     Squared Returns Ljung-Box Q Statistic                                   \
                                     Lag 1      Lag 2      Lag 3      Lag 4   
A                                 3.822511   4.150610   5.021057   5.086959   
AAL                               6.612910  10.316737  10.820808  11.255242   
AAPL                              0.366817   2.152137   6.341751   6.651563   
ABBV                             14.055561  47.373323  49.758082  50.483769   
ABT                               2.542660   2.602495   2.696170   2.828865   

                Absolute Returns Ljung-Box Q Statistic                        \
          Lag 5                                  Lag 1      Lag 2      Lag 3   
A      6.807568                               3.917258   3.929319   4.220997   
AAL   11.268415                               3.795328   7.650845   8.978590   
AAPL   7.094242                               3.539839   9.695576  12.072438   
ABBV  50.516932                               7.893987  47.057489  50.986263   
ABT    2.962482                              10.050528  10.147161  10.211366   

                            
          Lag 4      Lag 5  
A      5.233332   9.702744  
AAL    9.311429   9.882999  
AAPL  15.763208  16.972498  
ABBV  54.723432  56.102631  
ABT   10.286429  10.327349

In [22]:
#Autocorrelation Dataframe for different time lags
corr = pd.DataFrame(
    {
    **{("Returns Autocorrelation", f"Lag {l+1}"): r_ac[l] for l in range(5)},
    **{("Square Returns Autocorrelation", f"Lag {l+1}"): sqr_r_ac[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation", f"Lag {l+1}"): abs_r_ac[l] for l in range(5)},
    **{("Returns Partial Autocorrelation", f"Lag {l+1}"): r_pac[l] for l in range(5)},
    **{("Square Returns Partial Autocorrelation", f"Lag {l+1}"): sqr_r_pac[l] for l in range(5)},
    **{("Absolute Returns Partial Autocorrelation", f"Lag {l+1}"): abs_r_pac[l] for l in range(5)}
    }, index = returns.columns)
corr.head()

Returns Autocorrelation                                          \
                       Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
A                  -0.094774 -0.037028  0.045496 -0.005330 -0.089970   
AAL                 0.057755  0.006708  0.043228 -0.063062 -0.074242   
AAPL                0.074761 -0.011105 -0.083179 -0.052156  0.014010   
ABBV               -0.087293 -0.080589  0.119685 -0.035201 -0.092722   
ABT                -0.060655 -0.037864  0.070113  0.012528  0.008233   

     Square Returns Autocorrelation                                          \
                              Lag 1     Lag 2     Lag 3     Lag 4     Lag 5   
A                          0.122673 -0.035868 -0.058304  0.016010  0.081642   
AAL                        0.161350  0.120511  0.044368 -0.041107 -0.007143   
AAPL                       0.038001  0.083668  0.127913  0.034714  0.041411   
ABBV                       0.235232  0.361444  0.096505  0.053128  0.011334   
ABT                        0.100050 -0.015317 -0.019127 -0.022718 -0.022751   

      ... Square Returns Partial Autocorrelation                      \
      ...                                  Lag 1     Lag 2     Lag 3   
A     ...                               0.123163 -0.052116 -0.048776   
AAL   ...                               0.161995  0.097803  0.011724   
AAPL  ...                               0.038153  0.083005  0.124380   
ABBV  ...                               0.236173  0.326797 -0.044388   
ABT   ...                               0.100450 -0.025791 -0.015341   

                         Absolute Returns Partial Autocorrelation            \
         Lag 4     Lag 5                                    Lag 1     Lag 2   
A     0.028697  0.074725                                 0.124680 -0.008749   
AAL  -0.064145  0.002130                                 0.122724  0.110546   
AAPL  0.021206  0.020754                                 0.118522  0.144593   
ABBV -0.088049 -0.001999                                 0.176992  0.375454   
ABT  -0.020153 -0.019734                                 0.199711 -0.061978   

                                    
         Lag 3     Lag 4     Lag 5  
A    -0.034478  0.073618  0.119727  
AAL   0.047059 -0.064877  0.046590  
AAPL  0.066811  0.086687  0.028943  
ABBV  0.018324 -0.049567  0.014627  
ABT   0.034049 -0.030308  0.025748  

[5 rows x 30 columns]

In [23]:
#Number of significant p values in observations
sig_p = np.sum(p < 0.05, axis=0)
sig_p

Returns Autocorrelation P Value           Lag 1     48
                                          Lag 2     38
                                          Lag 3     35
                                          Lag 4     30
                                          Lag 5     33
Squared Returns Autocorrelation P Value   Lag 1    164
                                          Lag 2    174
                                          Lag 3    170
                                          Lag 4    175
                                          Lag 5    165
Absolute Returns Autocorrelation P Value  Lag 1    271
                                          Lag 2    295
                                          Lag 3    293
                                          Lag 4    292
                                          Lag 5    285
dtype: int64

In [24]:
def returns_autocorrelation(tickers: list[str], start:str, end:str, filename=None, header=None):
    """
    Returns three dataframes of autocorrelation, ljung-box q statistic, and p-value for returns
    of stocks in the list of tickers
    :param tickers: list of tickers
    :param start: start date to download data from
    :param end: end date to download data from
    :param filename: name of file to upload data to (optional)
    :param header: header configuration of the file (optional)
    :return: Three dataframes, autocorrelation, ljung-box q statistic, and p-value
    """
    data = yf.download(tickers, start=start, end=end)
    returns = np.log1p(data["Adj Close"].pct_change())
    returns = returns.dropna(axis=1, thresh=len(returns)-200)

    def ac_format(ac):
        lags = ac[0][1:6]
        rest = ac[1:3]
        return [lags] + list(rest)

    full_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna(), nlags=5, qstat=True)) for symbol in returns.columns))
    full_sqr_r_ac = np.array(list(ac_format(acf(returns[symbol].dropna()**2, nlags=5, qstat=True)) for symbol in returns.columns))
    full_abs_r_ac = np.array(list(ac_format(acf(abs(returns[symbol].dropna()), nlags=5, qstat=True)) for symbol in returns.columns))
    r_pac = np.array(list(pacf(returns[symbol].dropna(), nlags=5)[1:6] for symbol in returns.columns)).T
    sqr_r_pac = np.array(list(pacf(returns[symbol].dropna()**2, nlags=5)[1:6] for symbol in returns.columns)).T
    abs_r_pac = np.array(list(pacf(abs(returns[symbol].dropna()), nlags=5)[1:6] for symbol in returns.columns)).T
    
    r_ac = full_r_ac[:,0].T
    r_ldq = full_r_ac[:,1].T
    r_pval = full_r_ac[:,2].T
    sqr_r_ac = full_sqr_r_ac[:,0].T
    sqr_r_ldq = full_sqr_r_ac[:,1].T
    sqr_r_pval = full_sqr_r_ac[:,2].T
    abs_r_ac = full_abs_r_ac[:,0].T
    abs_r_ldq = full_abs_r_ac[:,1].T
    abs_r_pval = full_abs_r_ac[:,2].T

    symbols = returns.columns
    corr = pd.DataFrame(
    {
    **{("Returns Autocorrelation", f"Lag {l+1}"): r_ac[l] for l in range(5)},
    **{("Square Returns Autocorrelation", f"Lag {l+1}"): sqr_r_ac[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation", f"Lag {l+1}"): abs_r_ac[l] for l in range(5)},
    **{("Returns Partial Autocorrelation", f"Lag {l+1}"): r_pac[l] for l in range(5)},
    **{("Square Returns Partial Autocorrelation", f"Lag {l+1}"): sqr_r_pac[l] for l in range(5)},
    **{("Absolute Returns Partial Autocorrelation", f"Lag {l+1}"): abs_r_pac[l] for l in range(5)}
    }, index = returns.columns)

    ldq = pd.DataFrame(
    {
    **{("Returns Ljung-Box Q Statistic", f"Lag {l+1}"): r_ldq[l] for l in range(5)},
    **{("Squared Returns Ljung-Box Q Statistic", f"Lag {l+1}"): sqr_r_ldq[l] for l in range(5)},
    **{("Absolute Returns Ljung-Box Q Statistic", f"Lag {l+1}"): abs_r_ldq[l] for l in range(5)}
    }, index = returns.columns)

    p = pd.DataFrame(
    {
    **{("Returns Autocorrelation P Value", f"Lag {l+1}"): r_pval[l] for l in range(5)},
    **{("Squared Returns Autocorrelation P Value", f"Lag {l+1}"): sqr_r_pval[l] for l in range(5)},
    **{("Absolute Returns Autocorrelation P Value", f"Lag {l+1}"): abs_r_pval[l] for l in range(5)}
    }, index = returns.columns)

    if filename is not None:
        corr.to_csv("corr-"+filename, header=header)
        p.to_csv("p-"+filename, header=header)

    return corr, ldq, p

In [25]:
for year in range(2000, 2023):
    corr,_,p = returns_autocorrelation(list(symbols), f"{year}-01-01", f"{year}-12-31",\
        filename=f"rcd-{year}.csv")
    if year == 2000:
        measured_p = np.sum(p != np.nan, axis=0)
        sig_p = np.sum(p < 0.05, axis=0)
    else:
        measured_p += np.sum(p != np.nan, axis=0)
        sig_p += np.sum(p < 0.05, axis=0)

[*********************100%%**********************]  503 of 503 completed


133 Failed downloads:
['KHC', 'GPN', 'FSLR', 'CARR', 'CZR', 'HPE', 'NWSA', 'CTLT', 'LYV', 'PAYC', 'GOOG', 'INVH', 'FLT', 'ENPH', 'FTNT', 'TDG', 'FOXA', 'AIZ', 'MA', 'LKQ', 'CMG', 'ALLE', 'TRGP', 'CBRE', 'EXR', 'VICI', 'PYPL', 'TMUS', 'CTVA', 'PANW', 'BR', 'KVUE', 'LVS', 'DAL', 'ABBV', 'ZBH', 'XYL', 'CEG', 'BG', 'AWK', 'WYNN', 'CDAY', 'GM', 'NDAQ', 'KDP', 'MRNA', 'CFG', 'ZTS', 'WBD', 'CRM', 'SEDG', 'PRU', 'MDLZ', 'CF', 'AAL', 'ABNB', 'CME', 'APTV', 'CBOE', 'CDW', 'ULTA', 'ELV', 'IQV', 'NCLH', 'HWM', 'KEYS', 'HCA', 'AXON', 'MOH', 'VLTO', 'TEL', 'VRSK', 'EPAM', 'AMCR', 'DOW', 'LW', 'ACN', 'ICE', 'DXCM', 'MSCI', 'MKTX', 'FOX', 'CE', 'UAL', 'CNC', 'FTV', 'PODD', 'WTW', 'LDOS', 'GOOGL', 'NXPI', 'TSLA', 'PSX', 'BX', 'FANG', 'PM', 'AVGO', 'HLT', 'QRVO', 'EXPE', 'NWS', 'NRG', 'ETSY', 'PFG', 'GNRC', 'DPZ', 'MPC', 'ANET', 'SYF', 'NOW', 'FIS', 'DLR', 'PARA', 'CHTR', 'IR', 'DG', 'WRK', 'HII', 'STX', 'OTIS', 'LYB', 'ALGN', 'NFLX', 'META', 'AMP', 'V', 'DFS', 'LULU', 'GEHC', 'KMI', 'MPWR']: Exception

/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/1383132826.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Adj Close"].pct_change())


[*********************100%%**********************]  502 of 503 completed


120 Failed downloads:
['FSLR', 'KHC', 'CARR', 'NWSA', 'CZR', 'HPE', 'GOOG', 'CTLT', 'LYV', 'PAYC', 'INVH', 'ENPH', 'FLT', 'FTNT', 'TDG', 'FOXA', 'MA', 'AIZ', 'LKQ', 'CMG', 'ALLE', 'CBRE', 'TMUS', 'TRGP', 'EXR', 'PANW', 'VICI', 'PYPL', 'CTVA', 'BR', 'KVUE', 'LVS', 'DAL', 'ABBV', 'XYL', 'AWK', 'CEG', 'CDAY', 'WYNN', 'GM', 'MRNA', 'NDAQ', 'KDP', 'CFG', 'ZTS', 'WBD', 'CRM', 'SEDG', 'CF', 'AAL', 'ABNB', 'CME', 'APTV', 'CBOE', 'ULTA', 'CDW', 'IQV', 'NCLH', 'HWM', 'KEYS', 'HCA', 'TEL', 'VLTO', 'MOH', 'EPAM', 'VRSK', 'DOW', 'AMCR', 'LW', 'DXCM', 'ICE', 'MSCI', 'FOX', 'MKTX', 'CE', 'UAL', 'FTV', 'PODD', 'GOOGL', 'LDOS', 'NXPI', 'TSLA', 'PSX', 'BX', 'PM', 'FANG', 'HLT', 'AVGO', 'EXPE', 'QRVO', 'NWS', 'NRG', 'ETSY', 'GNRC', 'DPZ', 'MPC', 'ANET', 'SYF', 'NOW', 'PARA', 'DLR', 'CHTR', 'IR', 'DG', 'WRK', 'STX', 'HII', 'OTIS', 'NFLX', 'LYB', 'V', 'META', 'AMP', 'DFS', 'LULU', 'KMI', 'GEHC', 'MPWR']: Exception("%ticker%: Data doesn't exist for startDate = 978325200, endDate = 1009774800")
['BRK.B']: E

...


6 Failed downloads:
['KVUE', 'CEG', 'VLTO', 'GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 1609477200, endDate = 1640926800")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-01 -> 2021-12-31)')


/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/1383132826.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Adj Close"].pct_change())


[*********************100%%**********************]  503 of 503 completed


4 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1672462800")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-31)')


/var/folders/c0/_1r630sx6bx7dllfq1l9hgrh0000gn/T/ipykernel_53234/1383132826.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = np.log1p(data["Adj Close"].pct_change())


In [26]:
measured_p

Returns Autocorrelation P Value           Lag 1    10255
                                          Lag 2    10255
                                          Lag 3    10255
                                          Lag 4    10255
                                          Lag 5    10255
Squared Returns Autocorrelation P Value   Lag 1    10255
                                          Lag 2    10255
                                          Lag 3    10255
                                          Lag 4    10255
                                          Lag 5    10255
Absolute Returns Autocorrelation P Value  Lag 1    10255
                                          Lag 2    10255
                                          Lag 3    10255
                                          Lag 4    10255
                                          Lag 5    10255
dtype: int64

In [27]:
sig_p

Returns Autocorrelation P Value           Lag 1    1554
                                          Lag 2    1680
                                          Lag 3    1689
                                          Lag 4    1702
                                          Lag 5    1748
Squared Returns Autocorrelation P Value   Lag 1    2920
                                          Lag 2    3369
                                          Lag 3    3508
                                          Lag 4    3597
                                          Lag 5    3656
Absolute Returns Autocorrelation P Value  Lag 1    4038
                                          Lag 2    4488
                                          Lag 3    4645
                                          Lag 4    4802
                                          Lag 5    4815
dtype: int64

In [28]:
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display

In [29]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
def plot_sqr_returns_autocorrelation(ticker, transform, year, lags):
    match transform:
        case "Returns":
            t = lambda x: x
        case "Squared Returns":
            t = lambda x: x**2
        case "Absolute Returns":
            t = lambda x: abs(x)
    start = f"{year}-01-01"
    end = f"{year}-12-31"
    data = yf.download(ticker, start=start, end=end)
    returns = np.log1p(data["Adj Close"].pct_change())
    plot_acf(t(returns).dropna(), lags=lags)

In [30]:
ticker_text = widgets.Text(
    value="^GSPC",
    placeholder="Ticker Symbol"
)
year_text = widgets.BoundedIntText(
    value=2000,
    min=1970,
    max=2022
)
lags_text = widgets.BoundedIntText(
    value=5,
    min=3,
    max=100
)
transform_select = widgets.Select(options=["Returns", "Squared Returns", "Absolute Returns"])

In [31]:
_ = interact(plot_sqr_returns_autocorrelation,
            ticker = ticker_text,
            year = year_text,
            lags = lags_text,
            transform = transform_select)

interactive(children=(Text(value='^GSPC', description='ticker', placeholder='Ticker Symbol'), Select(descripti…